### Accidental millionaire 

This is a simple agent-based model as presented in the gitbook by Marco Janssen:

https://legacy.gitbook.com/book/cbie/introduction-to-agent-based-modeling/details

Agents predict the trend of the stock market and invest an equal amount in a shared pool. Agents who were correct, receive a share of the pool.

In [ ]:
import numpy as np

from mesa import Agent, Model
from mesa.time import RandomActivation

from bokeh.models import ColumnDataSource
from bokeh.plotting import output_notebook, figure, show
from bokeh.layouts import row
from bokeh.io import push_notebook

In [ ]:
#embed figures in the notebook
output_notebook()

### Creating the model

The evolution of the stock market is simulated by sampling a Bernouilli distribution (0 = stock market goes down, 1 = stock market goes up). The agents prediction of the stock market is modeled by sampling a Bernouilli distribution. In each step, all agents invest the same amount in the a shared pool. Each agent that made a correct prediction, receives an equal share of the pool.

In [ ]:
class StockPredictor(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.earnings = 0.0
        self.guess = None
    
    def step(self):
        self.guess = np.random.binomial(1,0.5)
        self.earnings -= self.model.stock_price

In [ ]:
class StockPredicterModel(Model):
    def __init__(self, N):
        self.number_agents = N
        self.stock_price = 1.0
        self.pool = 0.0
        self.stock_market = None
        self.schedule = RandomActivation(self)
        # Create agents
        for i in range(self.number_agents):
            a = StockPredictor(i, self)
            self.schedule.add(a)
            
    def step(self):
        self.pool = self.number_agents * self.stock_price
        self.stock_market = np.random.binomial(1, 0.5)
        self.schedule.step()
        self._payout()
    
    def _count_correct_agents(self):
        number_correct = 0
        for agent in self.schedule.agents:
            if agent.guess == self.stock_market:
                number_correct += 1
        return number_correct
    
    def _payout(self):
        number_correct = self._count_correct_agents()
        pool_share = self.pool / number_correct \
                        if number_correct > 0 else 0.0
        for agent in self.schedule.agents:
            if agent.guess == self.stock_market:
                agent.earnings += pool_share

#### Visualization 

These are functions for visualization in bokeh. We will visualize the histogram of the agents' earnings and the evolution of the maximum earning.

In [ ]:
def get_earnings(agents):
    return np.array([agent.earnings for agent in agents])

In [ ]:
def create_histogram_earnings(agents, number_bins=10):
    return np.histogram(get_earnings(agents), 
                        density=False, bins=number_bins)

In [ ]:
def max_earnings(agents):
    return np.max(get_earnings(agents))

In [ ]:
def update_max_earnings(old_data, agents, step_index):
    new_data = {}
    new_data['steps'] = np.append(old_data['steps'],[step_index]) \
                               if step_index else np.array([step_index])
    new_data['max_earnings'] = np.append(old_data['max_earnings'],
                                         [max_earnings(agents)]) \
                                    if step_index else np.array([max_earnings(agents)])
    return new_data

In [ ]:
def update_hist_earnings(agents, number_bins=10):
    new_data = {}
    hist, edges = create_histogram_earnings(agents, number_bins);
    new_data['hist'] = hist
    new_data['left_edges'] = edges[:-1]
    new_data['right_edges'] = edges[1:]

    return new_data

In [ ]:
def show_data(data_hist, data_max):
    fig_hist = figure(plot_width=350, plot_height=350, 
                      title='distribution earnings')
    fig_hist.quad(top='hist', bottom=0, left='left_edges', right='right_edges',
                  source=data_hist, fill_color="#036564", line_color="#033649")
    fig_max = figure(plot_width=640, plot_height=350,
                     title='maximum earnings')
    fig_max.line('steps', 'max_earnings', source=data_max, 
                  line_width=2, line_color='blue')
    fig_max.legend.location = "top_left"
    show(row(fig_hist, fig_max), notebook_handle=True)

### Run the model

In [ ]:
number_agents = 200
#Mesa is not so fast, so keep the number of agents reasonable
model = StockPredicterModel(number_agents) 

In [ ]:
data_hist = ColumnDataSource(
    data={'hist': [0.], 'left_edges': [0.], 'right_edges': [0.]})

data_max = ColumnDataSource(
    data={'steps': [0.], 'max_earnings': [0.]})

show_data(data_hist, data_max)

number_steps = 10000
number_bins = number_agents / 20
log_step = number_steps / 100
for i in range(number_steps):
    model.step()
    if not (i % log_step):
        data_hist.data = update_hist_earnings(
                                model.schedule.agents)
        data_max.data = update_max_earnings(
                                data_max.data, model.schedule.agents, i)
        push_notebook() 